In [70]:
EXPERIMENT_IDENTIFIER = "prompt_experiment_1"

In [4]:
import pandas as pd
ethnic_df = pd.read_csv("../data/ethnic_dataset_4893_after_removing_rakhain.csv")
ethnic_df

,Unnamed: 0,content
0,0,রমেল হত্যায় মানবাধিকার কমিশনের তদন্ত শুরুরাঙ্গ...
1,1,"নারায়ণগঞ্জের রূপগঞ্জ ও ফতুল্লা, গাজীপুরের কালি..."
2,2,গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগ...
3,3,চাঞ্চল্যকর পিলখানা হত্যা মামলায় ডেথ রেফারেন্স ...
4,4,অবাধে মাটি কেটে নেয়ার ধ্বংসের দ্বারপ্রান্তে ধো...
...,...,...
4888,4888,খাগড়াছড়ির দীঘিনালায় জেএসএস (এমএন লারমা) সমর্থি...
4889,4889,শান্তি-সম্প্রীতি স্থাপনে দুই দশক আগে পার্বত্য ...
4890,4890,জনপ্রিয় ম্যাগাজিন অনুষ্ঠান ‘ইত্যাদি’ আজ শুক্রব...
4891,4891,সরকারি বেতন স্কেলের নবম থেকে ১৩তম গ্রেড পর্যন্...


In [41]:
N = 5

In [17]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### Loading model

In [6]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 KB 6.5 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
!openai migrate

Error: Failed to fetch standard library grit module getgrit/stdlib: Failed to clone repo getgrit/stdlib: failed to connect to github.com: Connection timed out; class=Os (2)


In [35]:
pip install rich

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 KB 911.3 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 1.3 MB/s eta 0:00:001.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [71]:
from openai import OpenAI
import json

client = OpenAI()

def get_response(news_article):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a Bangla news article analyzer. I will give you a news article and you will say which genre(s) it belongs to. The genres are:  Politics, Crime, International Affairs,  Business & Economy,  Sports,  Culture & Entertainment,  Jobs,  Lifestyle,  Health,  Environment,  Local news,  Education,  Technology,  Religion,  Science,  Protests and Social Movements, Govt Actions.  Additionally, you will also say if the description is Straight, Investigation, Commentary. Identify the perspective of any ethnic person mentioned in the article, choosing from the following categories: Subject, Spokesperson, Expert or Commentator, Personal Experience, Popular Opinion, Eyewitness. Also, you will say whether this article is about Bangladeshi Ethnic People or not. Finally, you will say the sentiment of the article. Return me just a json of {\"genre\": GENRE, \"description_style\": STYLE, \"perspective\": PERSPECTIVE, \"is_ethnic\": True, \"sentiment\": \"positive\"}."},
            {"role": "user", "content": news_article}
        ]
    )
    return completion.choices[0].message.content
    
def post_process_chatgpt_turbo_3_5_response_to_json(text):
    json_string = text.replace("```", "").replace("json","").replace("\n","").replace("'", '"')
    json_output = json.loads(json_string)
    return json_output

In [72]:
from tqdm import tqdm

articles = ethnic_df["content"].values.tolist()
responses = []
for article in tqdm(articles[:N]):
    responses.append(get_response(article))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.11s/it]


In [73]:
responses

['```json\n{\n  "genre": ["Politics", "Crime", "Local news", "Protests and Social Movements"],\n  "description_style": "Straight",\n  "perspective": "Eyewitness",\n  "is_ethnic": true,\n  "sentiment": "negative"\n}\n```',
 '```json\n{\n  "genre": "Crime",\n  "description_style": "Straight",\n  "perspective": "Eyewitness",\n  "is_ethnic": true,\n  "sentiment": "negative"\n}\n```',
 '```json\n{\n  "genre": ["Crime", "Politics", "Local news"],\n  "description_style": "Straight",\n  "perspective": "Eyewitness",\n  "is_ethnic": true,\n  "sentiment": "negative"\n}\n```',
 '```json\n{\n  "genre": "Crime",\n  "description_style": "Straight",\n  "perspective": "Eyewitness",\n  "is_ethnic": false,\n  "sentiment": "neutral"\n}\n```',
 '```json\n{\n  "genre": ["Environment", "Local news", "Govt Actions"],\n  "description_style": "Straight",\n  "perspective": "Eyewitness",\n  "is_ethnic": true,\n  "sentiment": "negative"\n}\n```']

In [74]:
ethnic_sample_df = ethnic_df.iloc[:N]
ethnic_sample_df["Genre"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["genre"] for response in responses]
ethnic_sample_df["Description Style"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["description_style"] for response in responses]
ethnic_sample_df["Perspective"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["perspective"] for response in responses]
ethnic_sample_df["Is_ethnic"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["is_ethnic"] for response in responses]
ethnic_sample_df["Sentiment"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["sentiment"] for response in responses]

ethnic_sample_df.head()

/tmp/ipykernel_4231/2263428076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ethnic_sample_df["Genre"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["genre"] for response in responses]
/tmp/ipykernel_4231/2263428076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ethnic_sample_df["Description Style"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["description_style"] for response in responses]
/tmp/ipykernel_4231/2263428076.py:4: SettingWithCopyWarning: 
A value is try

,Unnamed: 0,content,Genre,Description Style,Perspective,Is_ethnic,Sentiment
0,0,রমেল হত্যায় মানবাধিকার কমিশনের তদন্ত শুরুরাঙ্গ...,"[Politics, Crime, Local news, Protests and Soc...",Straight,Eyewitness,True,negative
1,1,"নারায়ণগঞ্জের রূপগঞ্জ ও ফতুল্লা, গাজীপুরের কালি...",Crime,Straight,Eyewitness,True,negative
2,2,গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগ...,"[Crime, Politics, Local news]",Straight,Eyewitness,True,negative
3,3,চাঞ্চল্যকর পিলখানা হত্যা মামলায় ডেথ রেফারেন্স ...,Crime,Straight,Eyewitness,False,neutral
4,4,অবাধে মাটি কেটে নেয়ার ধ্বংসের দ্বারপ্রান্তে ধো...,"[Environment, Local news, Govt Actions]",Straight,Eyewitness,True,negative


In [75]:
ethnic_sample_df.to_csv(f"../data/ethnic_{N}_chatgpt4o_mini_{EXPERIMENT_IDENTIFIER}.csv")

In [76]:
import joblib
joblib.dump(responses, f"../data/ethnic_{N}_chatgpt4o_mini_{EXPERIMENT_IDENTIFIER}.csv")

['../data/ethnic_5_chatgpt4o_mini_prompt_experiment_1.csv']

In [77]:
ethnic_sample_df.iloc[4].values.tolist()

[np.int64(4),
 'অবাধে মাটি কেটে নেয়ার ধ্বংসের দ্বারপ্রান্তে ধোবাউড়া উপজেলার সিরামিকস পাহাড়। জানা গেছে, ১নং দক্ষিণ মাইজপাড়া ইউনিয়নে পুটিমারী বাজারের পাশেই অবস্থিত সিরামিকস পাহাড় স্থানীয় প্রভাবশালীদের সহায়তায় এক শ্রেণীর অসাধু চক্র পরিবেশ অধিদফতরের ছাড়পত্র না নিয়েই নির্বিচারে মাটি কেটে ধ্বংস করে দিচ্ছে। একই অবস্থা সেখানকার আদিবাসীদের বসতি গারো পাহাড়েও। প্রতিদিন শত শত ট্রাক সাদামাটি কেটে নিয়ে গ্রাস করছে প্রাকৃতিক এ সৌন্দর্যকে। প্রভাবশালী চক্র সংশ্লিষ্ট প্রশাসনকে ম্যানেজ করে তাদের রাজত্ব কায়েম করেছে। সরেজমিনে পরিদর্শন করে স্থানীয়দের সঙ্গে কথা বলে এমনই তথ্য পাওয়া গেছে। স্থানীয়দের অভিযোগ খনিজ সম্পদ ব্যুরো থেকে ১ হাজার বস্তা মাটি নেয়ার অনুমোদন এনে স্থানীয় প্রভাবশালীদের সহায়তায় হাজার হাজার বস্তা মাটি নিয়ে যায়। এ বিষয়ে প্রশাসনের কোনো দৃষ্টি নেই। তবে ২০১৪ সালের শেষের দিকে নতুন যোগদান করা উপজেলা নির্বাহী অফিসার মোহাম্মদ আনিসুজ্জমান খান সরেজমিনে পরিদর্শন করে মাটি কাটা সাময়িক বন্ধ করে দেন। এ ব্যাপারে তিনি একটি কমিটি গঠনও করে দেন। কিন্তু কিছুদিন বন্ধ থাকার পর আবারও শুরু হয়েছে মাটি কাটা। স্থানীয়রা জান

In [78]:
ethnic_sample_df = ethnic_df.sample(n=100, random_state=3)

In [79]:
ethnic_sample_df.to_csv("../data/annotated_100_ethnic_article_against_5_metrics.csv")